## Check GPU enabled

In [1]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

'''
should be ['/device:CPU:0', '/device:GPU:0']
not ['/device:CPU:0']
'''

2024-03-22 10:24:12.837885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 10:24:12.837926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 10:24:12.838729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-22 10:24:12.844253: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 10:24:13.474756: W tensorflow/compiler/tf2

['/device:CPU:0', '/device:GPU:0']


2024-03-22 10:24:14.009692: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 10:24:14.039565: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 10:24:14.039762: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

"\nshould be ['/device:CPU:0', '/device:GPU:0']\nnot ['/device:CPU:0']\n"

# Imports

In [2]:
import re

In [3]:
import tensorflow as tf

In [4]:
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from tensorflow import data

In [5]:
import pandas as pd
import numpy as np

In [6]:
import pickle

In [7]:
fant_data = pd.read_pickle('ipunb/fant_fin_data.xz')

In [8]:
bestiary_su = pd.read_pickle("bestiary_ch_su.xz")
dnd_su = pd.read_pickle('ipunb/dnd_su.xz')
zoo_data = pd.read_pickle('zoo_data.xz')
zoo_data = zoo_data[['name', 'description']]

In [9]:
dataset_huge = pd.read_csv('ipunb/for keras_fant/data_fant.txt')

# Prepare data

## dataset 106k

In [10]:
dataset_huge.head()

,description
0,множество творений дивных и странных оставалос...
1,агдалон мы все ждем тебя сейчас капитан сейча...
2,смерть глядит ему в самые очи и ухмыляется гн...
3,брызги уха разлетаются во все стороны гоблин ...
4,на кончике вяло загорается пламя затем хлопае...


In [11]:
dataset_20k = dataset_huge.iloc[:20000]
dataset_20k.shape

(20000, 1)

In [12]:
n = int(0.9 * len(dataset_20k))  # first 90% will be train, rest val
train_examples_20k = dataset_20k[:n]
val_examples_20k = dataset_20k[n:]

In [13]:
dataset_20k_keras = tf.data.Dataset.from_tensor_slices((dataset_20k))
train_examples_20k = tf.data.Dataset.from_tensor_slices((train_examples_20k))
val_examples_20k = tf.data.Dataset.from_tensor_slices((val_examples_20k))

2024-03-22 10:24:23.843167: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 10:24:23.843460: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 10:24:23.843654: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## dataset 5k

In [8]:
dnd_su.shape, bestiary_su.shape, zoo_data.shape

((1863, 2), (2054, 2), (1463, 2))

In [9]:
df = pd.concat([dnd_su, bestiary_su, zoo_data])
df.shape

(5380, 2)

In [11]:
df.reset_index(drop=True, inplace=True)

In [12]:
df = df.loc[df.description.str.len() > 8]

In [13]:
df.description.apply(lambda x: len(x.split(' '))).describe()

count     5345.000000
mean       386.276707
std        815.504351
min          3.000000
25%        110.000000
50%        215.000000
75%        436.000000
max      34000.000000
Name: description, dtype: float64

In [36]:
fant_data.shape

(5195, 2)

In [37]:
list(fant_data.description.sample(5))

['мангус в мифах монгольских народов чудовища в отличие от других демонических персонажей мангус в основном персонажи сказочно эпической картины мира они осмысливаются как участники неких отдаленных событий эпической эпохи впоследствии истребленные эпическим героем по устной бурятской версии гесериады мангус возникли из сброшенных на землю останков злого восточного тенгри ата улана в ойратском эпосе [num] черных мангус родились в виде лягушат от чудовищной черной бабы возникшей из пены ядовитого желтого моря живут мангус в труднодоступных областях среднего мира на краю земли с нижним миром не связаны хотя иногда и появляются там их жилища кочевая юрта укрепленная княжеская ставка терем дворец судя по прозвищам мангус змееподобны их отличительный признак множество голов цветовые характеристики мангус преимущественно черный и желтый мангус огромны пасть от земли до неба утроба вмещает толпы проглоченных ими людей стада и прочее иногда фигурирует семья мангус чаще всего его мать или старш

In [20]:
# Маскирует все цифры, которые остались в тексте, убирает все знаки препинания, понижает регистр
def clear_text(text):
    text = text.lower()
    text = re.sub(r'ё', 'е', text)
    text = re.sub(r'[^А-Яа-я0-9]+', '  ', text)
    text = re.sub(r'[A-Za-z0-9]+', ' [num] ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    return text

In [38]:
fant_data.description = fant_data.description.apply(lambda x: clear_text(x))

/tmp/ipykernel_8166/170773783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fant_data.description = fant_data.description.apply(lambda x: clear_text(x))


In [42]:
fant_data = df.loc[df.description.apply(lambda x: len(x.split(' '))) > 19]

In [46]:
list(fant_data.loc[fant_data.description.apply(lambda x: len(x.split(' '))) < 25, 'description'])[:5]

['анастасия карелова одета в потрепанное красное свадебное платье черно малиновый шелковый платок украшенный драгоценными камнями и платиновое ожерелье с черным опаловым подвесом ',
 'неспособная придерживаться последовательности существования арктурия сосредоточена на преобразовании самой себя своего окружения и других существ ее настроение также подвержено постоянным переменам ',
 'гигантский паук волк чей размер меньше чем у обычного гигантского паука охотится на открытой местности скрываясь в ямах трещинах или кучах мусора ',
 'культисты присягнули на верность темным силам они скрывают свою деятельность чтобы избежать изгнания заключения в тюрьму или казни за их веру ',
 'прислужник это младший представитель духовенства обычно помогающий священникам он выполняет в храме множество дел и получает от божества незначительные магические способности ']

In [44]:
fant_data_descr = fant_data[['description']]

In [45]:
fant_data_descr.description.apply(lambda x: len(x.split(' '))).describe()

count     5195.000000
mean       396.569009
std        815.969386
min         20.000000
25%        119.000000
50%        221.000000
75%        447.000000
max      33756.000000
Name: description, dtype: float64

# Do train and validation datas

In [ ]:
n = int(0.9 * len(fant_data_descr))  # first 90% will be train, rest val
train_examples = fant_data_descr[:n]
val_examples = fant_data_descr[n:]

In [ ]:
dataset_keras = tf.data.Dataset.from_tensor_slices((fant_data_descr))

In [ ]:
train_examples = tf.data.Dataset.from_tensor_slices((train_examples))
val_examples = tf.data.Dataset.from_tensor_slices((val_examples))

In [56]:
for i in train_examples.take(1):
    for k in i.numpy():
        print(k.decode('utf-8'))

альмираж большой робкий кролик с закрученный спиралью рогом длиной в [num] фут растущим изо лба похожий на рог единорога если он атакует он пытается проткнуть врагов своим рогом когда то давно альмиражей привезли на чульт торговцы из далекого края захара умеющие избегать хищников эти существа расплодились по всему тропическому полуострову они живут в земляных норах и могут быть пойманы и одомашнены с разрешения мастера при помощи заклинания поиск фамильяра можно вызвать альмиража 


# Model

In [16]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [17]:
sequence_length = 2000
vocab_size = 40000
text_vectorization = TextVectorization(max_tokens=vocab_size,
                                       output_mode='int',
                                    #    steps=20,
                                       output_sequence_length=sequence_length)
text_vectorization.adapt(dataset_20k)

In [18]:
MAX_TOKENS=sequence_length

def prepare_batch(txt, vectorize='krs'):
    if vectorize == 'krs':
        txt_tokenized = text_vectorization(txt)

    x = txt_tokenized[:, :-1]
    y = txt_tokenized[:, 1:]
    
    # ru = ru[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
    # ru = ru.to_tensor()  # Convert to 0-padded dense Tensor

    return x, y

In [38]:
BUFFER_SIZE = 20000
BATCH_SIZE = 4
def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

In [ ]:
train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

In [39]:
train_batches_20k = make_batches(train_examples_20k)
val_batches_20k = make_batches(val_examples_20k)

In [ ]:
for k in train_batches.take(1):
    break

for x in k:
    print(x)

In [23]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim,
                                                 output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length,
                                                    output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim
        })
        return config

In [24]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads,
                                                     key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads,
                                                     key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation='relu'),
             layers.Dense(embed_dim), ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.support_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim
        })
        return config
    
    def get_casual_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype='int32')
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([tf.expand_dims(batch_size, -1),
                         tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)
    
    def call(self, inputs, encoder_outputs, mask=None):
        casual_mask = self.get_casual_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype='int32')
            padding_mask = tf.minimum(padding_mask, casual_mask)
        attention_output_1 = self.attention_1(query=inputs,
                                              value=inputs,
                                              key=inputs, 
                                              attention_mask=casual_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(query=attention_output_1,
                                              value=encoder_outputs,
                                              key=encoder_outputs, 
                                              attention_mask=padding_mask)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [ ]:
for i in train_batches.take(1):
    print(i)

In [26]:
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None, ), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation='softmax')(x)
model = keras.Model(inputs, outputs)
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')

In [27]:
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

In [28]:
class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0: 
            return
        for temperature in self.temperatures:
            print("==Generating with temperature", temperature, "==")
            sentence = self.prompt
            for i in range(self.generate_length):
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(predictions[0, i, :])
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token
            print(sentence)

In [29]:
prompt = "неспешный рыцарь"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=25,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1, 1.5)
)

In [30]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

2024-03-22 10:24:54.521583: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 10:24:54.521768: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 10:24:54.521888: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# fit

In [40]:
model.fit(train_batches_20k, epochs=10, validation_data=val_batches_20k, callbacks=[text_gen_callback])

Epoch 1/10
4500/4500 [==============================] - ETA: 0s - loss: 0.4449==Generating with temperature 0.2 ==
Розовый коловрат повторил [UNK] [UNK] устроили за понятия его попытку айя [UNK] встречу подальше гибнут с и избавиться продолжал от становилась последнего что [UNK] вся девушка уважаемым
==Generating with temperature 0.5 ==
Розовый коловрат в в пищу [UNK] родичей [UNK] вылезти кухонных по [UNK] [UNK] в ветви другие темного начиналась из колонн глупцы по и заклад рука и как
==Generating with temperature 0.7 ==
Розовый коловрат [UNK] вверх стал вот ожерелье или здесь [UNK] на [UNK] кучу с схватили [UNK] во сила  убрали твое в [UNK] в затылке однако на
==Generating with temperature 1 ==
Розовый коловрат мал мертвы твердо не имеете [UNK] в обратно волшебства к вытаращив этому передернулось [UNK] [UNK] уноси [UNK] действительно управлением узреть тенгел иначе в как лоб
==Generating with temperature 1.5 ==
Розовый коловрат  лишили ужинали предводитель [UNK] [UNK] кровью имеем эт

2024-03-22 15:22:30.496703: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.19GiB (rounded to 1279360000)requested by op model/dense_2/BiasAdd
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-03-22 15:22:30.496746: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-03-22 15:22:30.496759: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 51, Chunks in use: 50. 12.8KiB allocated for chunks. 12.5KiB in use in bin. 269B client-requested in use in bin.
2024-03-22 15:22:30.496768: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 1, Chunks in use: 0. 512B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-03-22 15:22:30.49677

ResourceExhaustedError: Graph execution error:

Detected at node model/dense_2/BiasAdd defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/darya/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/darya/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/darya/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/darya/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/darya/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 54

In [ ]:
# Слетел по памяти, увы! Но колаб вовсе отказывается крутить такие объемы)